# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

import os
import logging

# Set logging level to suppress warnings
logging.getLogger().setLevel(logging.ERROR)
# Just disables the warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from nn.models_tf2 import Models
from botbidder import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'), False)  # Load sampling strategies


Loading config file d:\GitHub\ben\src\./config/default.conf
Loading config file d:\GitHub\ben\src\./config/default.conf


### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

This script was used to detect the reason for some strange bidding. But the error is fixed, so this should look normal now

In [2]:
hands = ['T8732.K7532.J.Q3', '4.AQJ98.KQ975.94', 'AQ.T6.A863.JT862', 'KJ965.4.T42.AK75']

In [3]:
from ddsolver import ddsolver
dds = ddsolver.DDSolver()

bidder_bots = [BotBid([True, False], hand, models, sampler, i, 3, dds, False, False) for i, hand in enumerate(hands)]

In [4]:
auction = ['PAD_START','PAD_START','PAD_START']  # since West deals we need 'PAD_START'

turn_i = 3 # whose turn is it to bid
step = 0

while not bidding.auction_over(auction):
    bid = bidder_bots[turn_i].bid(auction)
    print(bid.to_dict())
    auction.append(bid.bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    step += 1
print(auction)

{'bid': '1S', 'who': 'NN', 'quality': '1', 'candidates': [{'call': '1S', 'insta_score': 1.196, 'alert': 'False', 'explanation': '5+ !S -- 6-!H; 5+!S; 11-20 HCP'}], 'hcp': [9.7, 9.8, 9.3], 'shape': [2.8, 4.0, 3.1, 3.0, 2.9, 3.9, 3.1, 2.9, 2.8, 3.9, 3.1, 3.0], 'explanation': '5+ !S -- 6-!H; 5+!S; 11-20 HCP'}
{'bid': 'PASS', 'who': 'NN', 'quality': '1', 'candidates': [{'call': 'PASS', 'insta_score': 1.2, 'alert': 'False', 'explanation': ' -- ; 14- HCP'}], 'hcp': [11.0, 9.3, 13.6], 'shape': [1.7, 3.0, 4.2, 4.0, 1.6, 3.1, 4.2, 4.0, 5.1, 2.1, 2.8, 2.9], 'explanation': ' -- ; 14- HCP'}
{'bid': '1N', 'who': 'NN', 'quality': '1', 'candidates': [{'call': '1N', 'insta_score': 1.068, 'alert': 'True', 'explanation': 'Forcing 1NT -- 3-!S; 5-12 HCP; Artificial; Forcing'}], 'hcp': [7.7, 13.1, 6.4], 'shape': [3.0, 2.9, 2.9, 4.0, 5.3, 2.3, 2.2, 3.2, 3.2, 2.9, 3.0, 3.9], 'alert': 'True', 'explanation': 'Forcing 1NT -- 3-!S; 5-12 HCP; Artificial; Forcing'}
{'bid': 'PASS', 'who': 'NN', 'quality': '1', 'can